In [4]:
import os	
import glob
import numpy as np
import geopandas as gpd
import rasterio
import rasterio.mask
from shapely.geometry import mapping
from scipy.stats import linregress
import matplotlib.pyplot as plt

# -------------------------------------------------------------
# 1. SETTINGS
# -------------------------------------------------------------
folder = r"C:\Users\DELL"
malawi_shp = os.path.join(folder, "Malawi.shp")
output_trend_png = os.path.join(folder, "Malawi_Rainfall_Trend_Map.png")

# -------------------------------------------------------------
# 2. FIND CHIRPS RAINFALL TIF FILES
# -------------------------------------------------------------
tif_list = sorted(glob.glob(os.path.join(folder, "chirps-v3.0*.tif")))
print("CHIRPS files found:")
for t in tif_list:
    print("  ", t)

if len(tif_list) == 0:
    raise ValueError("No CHIRPS tif files found in folder.")

# Extract years from filenames
years = []
for f in tif_list:
    digits = ''.join([c for c in os.path.basename(f) if c.isdigit()])
    years.append(int(digits))

print("Years detected:", years)

years_arr = np.array(years)

# -------------------------------------------------------------
# 3. LOAD MALAWI BOUNDARY
# -------------------------------------------------------------
mw = gpd.read_file(malawi_shp)
mw = mw.to_crs("EPSG:4326")   # enforce lat/lon CRS
mw_union = mw.unary_union     # combined polygon
mw_geom = [mapping(mw_union)] # required format for rasterio.mask

# -------------------------------------------------------------
# 4. LOAD AND CLIP RASTERS USING rasterio.mask
# -------------------------------------------------------------
rasters = []

for f in tif_list:
    print("Processing:", f)
    with rasterio.open(f) as src:
        clipped_array, clipped_transform = rasterio.mask.mask(
            src,
            mw_geom,
            crop=True
        )
        clipped_array = clipped_array[0]  # remove band dimension

        rasters.append(clipped_array)

# Stack into array: shape -> (time, height, width)
rain_stack = np.stack(rasters, axis=0)

print("Stacked rainfall array shape:", rain_stack.shape)

# -------------------------------------------------------------
# 5. COMPUTE PIXEL-WISE RAINFALL TREND (mm/year)
# -------------------------------------------------------------
print("Computing rainfall trend (mm/year)...")

n_years, ny, nx = rain_stack.shape
slope = np.full((ny, nx), np.nan)

for j in range(ny):
    for i in range(nx):
        y = rain_stack[:, j, i]
        if np.all(np.isnan(y)):
            continue
        lr = linregress(years_arr, y)
        slope[j, i] = lr.slope   # mm/year

# -------------------------------------------------------------
# 6. PLOT TREND MAP
# -------------------------------------------------------------
print("Saving rainfall trend map...")

plt.figure(figsize=(10, 10))
vmax = np.nanpercentile(np.abs(slope), 98)  # symmetric color scaling
plt.imshow(slope, cmap="RdBu", vmin=-vmax, vmax=vmax)
plt.colorbar(label="Rainfall Trend (mm/year)")
plt.title("Rainfall Trend Across Malawi (Based on Available CHIRPS TIFFs)")
plt.axis("off")
plt.savefig(output_trend_png, dpi=300, bbox_inches="tight")
plt.close()

print("Trend map saved to:", output_trend_png)

# -------------------------------------------------------------
# 7. SUMMARY STATISTICS
# -------------------------------------------------------------
valid = slope[~np.isnan(slope)]
national_mean = np.mean(valid)
max_increase = valid.max()
max_decrease = valid.min()

print("--------------------------------------------------")
print("RAINFA LL TREND SUMMARY (mm/year)")
print("National average trend:", national_mean)
print("Maximum increase:", max_increase)
print("Maximum decrease:", max_decrease)
print("--------------------------------------------------")


CHIRPS files found:
   C:\Users\DELL\chirps-v3.0.2000.tif
   C:\Users\DELL\chirps-v3.0.2005.tif
   C:\Users\DELL\chirps-v3.0.2010.tif
   C:\Users\DELL\chirps-v3.0.2015.tif
   C:\Users\DELL\chirps-v3.0.2020.tif
   C:\Users\DELL\chirps-v3.0.2024.tif
Years detected: [302000, 302005, 302010, 302015, 302020, 302024]


C:\Users\DELL\AppData\Local\Temp\ipykernel_2764\683043381.py:44: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  mw_union = mw.unary_union     # combined polygon


Processing: C:\Users\DELL\chirps-v3.0.2000.tif
Processing: C:\Users\DELL\chirps-v3.0.2005.tif
Processing: C:\Users\DELL\chirps-v3.0.2010.tif
Processing: C:\Users\DELL\chirps-v3.0.2015.tif
Processing: C:\Users\DELL\chirps-v3.0.2020.tif
Processing: C:\Users\DELL\chirps-v3.0.2024.tif
Stacked rainfall array shape: (6, 156, 66)
Computing rainfall trend (mm/year)...
Saving rainfall trend map...
Trend map saved to: C:\Users\DELL\Malawi_Rainfall_Trend_Map.png
--------------------------------------------------
RAINFA LL TREND SUMMARY (mm/year)
National average trend: -1.1982558981082754
Maximum increase: 13.95551521547379
Maximum decrease: -20.628936078471522
--------------------------------------------------
